In [39]:
import os
import numpy as np
import pandas as pd

import pymatgen as pg


In [40]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [41]:

def local_min(df):

    df['dy'] = df['Edis'].diff()

    df['sign_dy'] = np.sign(df['dy'])
    df['sign_change'] = df['sign_dy'].diff()

    # Indices of local maxima: derivative goes from negative (-1) to positive (+1)
    local_min_indices = df.index[(df['sign_change'] > 0)]

    local_mins = df.loc[local_min_indices, ['cell_length', 'Edis']].reset_index(drop=True)

    return local_mins

def local_max(df):
    df['dy'] = df['Edis'].diff()

    df['sign_dy'] = np.sign(df['dy'])
    df['sign_change'] = df['sign_dy'].diff()

    # Indices of local maxima: derivative goes from negative (-1) to positive (+1)
    local_min_indices = df.index[(df['sign_change'] < 0)]

    local_max = df.loc[local_min_indices, ['cell_length', 'Edis']].reset_index(drop=True)

    return local_max

In [42]:
def finding_diff(df):
    abs_max = df['Edis'].max()
    max_val = local_max(df)
    min_val = local_min(df)

    diff = (abs_max + max_val['Edis'][0])/2 - min_val['Edis'][0]

    return diff

In [43]:
from pymatgen.core import Structure

In [44]:
#list all folders inside li_d3s
li_d3s = os.listdir("li_d3s")

results = []

df = pd.DataFrame()
cnt=0
for items in li_d3s:
    cnt+=1
    #get the path of each folder
    path = os.path.join("li_d3s", items)
    # print(items)
    
    #ignore .DS_Store
    if items == ".DS_Store":
        continue
    file = os.path.join(path, f'{items}.csv')
    struct_file = os.path.join(path, 'POSCAR')
    df = pd.read_csv(file)
    print(f"Reading {file}")

    struct = Structure.from_file(struct_file)
    sites = struct.num_sites

    # print(struct.composition['Li'])
    li_sites = struct.composition['Li']
   
    diff_energy = finding_diff(df)
    print(f'Composition {struct.composition}')
    print(f'diff per toal sites {diff_energy/sites}')
    print(f'diff per Li sites {diff_energy/li_sites}')
    print('-----------------------------------')

    result = {
        "Structure": struct.composition,
        "Total Sites": sites,
        "Li Sites": li_sites,
        "Diff Energy": diff_energy,
        "Diff per Total Sites": diff_energy/sites,
        "Diff per Li Sites": diff_energy/li_sites,
        "Composition": str(struct.composition)
    }

    results.append(result)
print(f'total structure analysed = {cnt}')
    

Reading li_d3s/LiGe_141/LiGe_141.csv
Composition Li6 Ge6
diff per toal sites 0.0013341615897780837
diff per Li sites 0.0026683231795561675
-----------------------------------
Reading li_d3s/LiMg_5/LiMg_5.csv
Composition Li3 Mg3
diff per toal sites 0.016657339849104266
diff per Li sites 0.03331467969820853
-----------------------------------
Reading li_d3s/LiB_227/LiB_227.csv
Composition Li2 B2
diff per toal sites 0.0010879295338342499
diff per Li sites 0.0021758590676684997
-----------------------------------
Reading li_d3s/LiSi_227/LiSi_227.csv
Composition Li2 Si2
diff per toal sites 0.001197946015120837
diff per Li sites 0.002395892030241674
-----------------------------------
Reading li_d3s/LiCl_225/LiCl_225.csv
Composition Li1 Cl1
diff per toal sites 0.00696345717606865
diff per Li sites 0.0139269143521373
-----------------------------------
Reading li_d3s/LiMn_187/LiMn_187.csv
Composition Li1 Mn1
diff per toal sites 0.0031490046097577247
diff per Li sites 0.0062980092195154495
---

In [ ]:
results_df = pd.DataFrame(results)
# results_df.to_csv("analysis_results.csv", index=False)

print(f"Total str.uctures analysed = {cnt}")
# print("Results saved to analysis_results.csv")

Total structures analysed = 73
Results saved to analysis_results.csv
